# 01. 피드 만들기

## 피드
- 발신 전용 데이터

## RSS
- RSS 요소
    - channel : 사이트 정보  
        1. 필수
        - title : 피드의 이름
        - link : 피드를 제공하는 서비스의 url
        - description : 피드의 설명  
        2. 옵션 
        - lastBuildDate : 피드의 최종 변경일
        - ttl : time to line으로 피드가 어느 정도 시간 후에 변경될지를 나타냄, 분 단위
        - image : 피드의 섬네일을 지정하며 url(이미지 주소), title, link의 필수 요소가 있음
    - item : channel 요소 계층 내부에 기사, 문서 , 상품 등의 아이템을 작성
        - title : 아이템의 제목
        - link : 아이템 상세 페이지
        - description : 아이템의 설명을 위한 요소
        - guid : 아이템을 유일하게 특정할 수 있는 문자열
        - pubDate : 아이템의 공개일

## 네임 스페이스를 사용한 RSS 확장
- `<publisher>` 같은 독자적으로 요소를 추가하고 싶다면 XML 네임 스페이스를 추가한 후에 사용해야 함
    - `<rss version="2.0" xmlns:book="http://my-service.com/xmlns/book" >` 등
    - Namespaces in XML 1.0 (Third Edition) - http://www.w3.org/TR/REC-xml-names/
    
- 사용자 정의 요소를 만들 수도 있음 `<book:publisher id="2">위키북스</book:publisher>`

### 마크업 작성을 포함할 때의 주의사항
- 문자열 값에 HTML 등의 마크업 문서가 포함돼 있으면 태그 기호가 XML 태그와 혼동되어 제대로 해석되지 않으므로 이스케이프 처리해줘야함
    - `<a href="http://example.com">` > `&lt;a href=&quot;http://example.com&quot;&gt;`
    - `%`, `<`, `>`, `"`, `'` > `&amp;`, `&lt;`, `&gt;`, `&quot;`, `&apos;` 사용

### 표준 XML 모듈을 사용해서 RSS 만들기

In [1]:
import xml.etree.ElementTree as ET
from xml.dom import minidom

# 사용자 정의 네임 스페이스
NAMESPACES = {'book':'http://my-service.com/xmlns/book'}

def create_rss():
    """RSS 만들기"""
    for ns_name, ns_url in NAMESPACES.items():
        ET.register_namespace(ns_name, ns_url) # 네임 스페이스 등록, url이 없어도 되며 rss에 book을 등록하기 위한 코드
        
    # <rss> 요소 만들기
    # <rss version="2.0" xmlns:book="http://my-service.com/xmlns/book"></rss>의 XML 객체 생성
    elm_rss = ET.Element(
        "rss",
        attrib={
            'version':"2.0",
            'xmlns:book':NAMESPACES['book']
        },
    )
    
    # <channel> 요소 만들기
    elm_channel = ET.SubElement(elm_rss, 'channel')
    
    # channel 요소의 서브 요소를 한 번에 추가하기(필수인 title, link, description)
    channel_sources = {
        'title':"위키북스의 도서 목록",
        'link':"http://example.com",
        'description':"설명 입력",
    }
    children_of_channel = []
    for tag, text in channel_sources.items():
        child_elm_of_channel = ET.Element(tag)
        child_elm_of_channel.text = text
        children_of_channel.append(child_elm_of_channel)
        
    # 한 번에 추가하기
    elm_channel.extend(children_of_channel)
    
    
    
    # <item> 요소 추가하기 : 하나씩 서브 요소 추가해보기
    elm_item = ET.SubElement(elm_channel, 'item')
    
    # <item><title> 요소 추가하기
    elm_item_title = ET.SubElement(elm_item, 'title')
    elm_item_title.text = "파이썬을 이용한 머신러닝, 딥러닝 실전 앱 개발"
    
    # <item><link> 요소 추가하기
    elm_item_link = ET.SubElement(elm_item, 'link')
    elm_item_link.text = "http://example.com"
    
    # <item><description> 요소 추가하기
    elm_item_description = ET.SubElement(elm_item, 'description')
    elm_item_description.text \
        = ( '<a href="http://example.com">이스케이프 처리 확인 전용 링크</a>'
           "설명 입력")
    
    # <item><book:publisher> 요소 추가하기
    elm_item_publisher = ET.SubElement(elm_item, 'book:publisher', id="1")
    elm_item_publisher.text = '위키북스'
    
    # XML 문자열로 변환하기
    xml = ET.tostring(elm_rss, 'utf-8')
    
    # 앞에 <?xml version="1.0"?>을 추가하고 보기 좋게 가공하기
    with minidom.parseString(xml) as dom:
        return dom.toprettyxml(indent='    ')
    
if __name__ == '__main__':
    rss_str = create_rss()
    print(rss_str)

<?xml version="1.0" ?>
<rss version="2.0" xmlns:book="http://my-service.com/xmlns/book">
    <channel>
        <title>위키북스의 도서 목록</title>
        <link>http://example.com</link>
        <description>설명 입력</description>
        <item>
            <title>파이썬을 이용한 머신러닝, 딥러닝 실전 앱 개발</title>
            <link>http://example.com</link>
            <description>&lt;a href=&quot;http://example.com&quot;&gt;이스케이프 처리 확인 전용 링크&lt;/a&gt;설명 입력</description>
            <book:publisher id="1">위키북스</book:publisher>
        </item>
    </channel>
</rss>



## feedgen을 사용해서 RSS 만들기
- xml 모듈을 이용하면 반복작업과 오탈자를 내도 무엇이 문제인지 알기 힘듬
- feedgen 라이브러리를 이용하면 좀 더 편함

### 네임 스페이스 전용 클래스 만들기
- `feedgen_ext.py`

```python
from lxml import etree
from feedgen.ext.base import BaseExtension

class BookBaseExtension(BaseExtension):
    """book 확장 네임 스페이스"""
    
    # 사용자 정의 네임 스페이스의 URL
    BOOK_NS = 'http://my-service.com/xmlns/book'
    
    def __init__(self):
        """사용자 정의 요소 이름 앞에는 __를 넣음"""
        # 변수는 자료형으로
        self.__publisher = {}
        
    def extend_ns(self):
        """확장 네임 스페이스"""
        return {'book':self.BOOK_NS}
    
    def _extend_xml(self, elm):
        """요소 추가하기"""
        if self.__publisher:
            publihser = etree.SubElement(
                elm, # 부모 요소
                '{%s}publisher' % self.BOOK_NS, # {<네임 스페이스의 URL>}요소 이름
                attrib={'id':self.__publisher.get('id')} # id 속성 적용
            )
            publisher.text = self.__publisher.get('name') # 요소의 내용 적용
        return elm
    
    def publisher(self, name_and_id_dict=None):
        """self.__publisher"""
        if name_and_id_dict is not None:
            name = name_and_id_dict.get('name')
            id_ = name_and_id_dict.get('id')
            if name and id_:
                self.__publisher = {'name':name, 'id':id_}
            elif not name and not id_: # name이 없는 경우는 요소를 만들지 않음
                self.__publisher = {}
            else:
                raise ValueError('name과 id를 함께 지정해주세요.')
        return self.__publisher
    
    
class BookFeedExtension(BookBaseExtension):
    """channel 요소에 적용"""
    
    def extend_rss(self, rss_feed):
        """요소 추가 때 호출"""
        channel = rss_feed[0]
        self._extend_xml(channel)
    
    
class BookEntryExtension(BookBaseExtension):
    """item 요소에 적용"""
    
    def extend_rss(self, entry):
        """요소 추가 때 호출"""
        self._extend_xml(entry)
```

### 피드를 생성하는 프로그램 만들기

In [2]:
from feedgen.feed import FeedGenerator

# 사용자 정의 네임 스페이스 클래스 읽어 들이기
from feedgen_ext import BookEntryExtension, BookFeedExtension

def create_feed():
    """RSS 피드 생성하기"""
    
    # 피드 데이터 저장 전용 객체
    fg = FeedGenerator()
    
    # 사용자 정의 네임 스페이스를 등록하고 이전에 만들었던 클래스 적용하기
    fg.register_extension(
        'book',
        extension_class_feed=BookFeedExtension,
        extension_class_entry=BookEntryExtension,
    )
    
    # <channel><title> 요소
    fg.title("위키북스의 도서 목록")
    
    # <channel><link> 요소 : href 속성으로 지정
    fg.link(href='http://example.com')
    
    # <channel><description> 요소
    fg.description("설명 입력")
    
    
    # <channel><item> 요소
    fe = fg.add_entry()
    
    # <title> 요소
    fe.title("파이썬을 이용한 머신러닝, 딥러닝 실전 앱 개발")
    
    # <link> 요소
    fe.link(href="http://example.com")
    
    # <description> 요소
    fe.description(
        '<a href="http://example.com">이스케이프 처리 확인 전용 링크</a>'
        "설명 입력")
    
    # <book:writer> 요소(사용자 정의 네임 스페이스)
    fe.book.publisher({'name':'위키북스', 'id':'1'})
    
    # 피드를 RSS 형식으로 변환(pretty=True로 들여쓰기 적용)
    return fg.rss_str(pretty=True)

if __name__ == '__main__':
    rss_str = create_feed()
    print(rss_str.decode())

<?xml version='1.0' encoding='UTF-8'?>
<rss xmlns:book="http://my-service.com/xmlns/book" xmlns:atom="http://www.w3.org/2005/Atom" xmlns:content="http://purl.org/rss/1.0/modules/content/" version="2.0">
  <channel>
    <title>위키북스의 도서 목록</title>
    <link>http://example.com</link>
    <description>설명 입력</description>
    <docs>http://www.rssboard.org/rss-specification</docs>
    <generator>python-feedgen</generator>
    <lastBuildDate>Thu, 07 Oct 2021 01:49:00 +0000</lastBuildDate>
    <item>
      <title>파이썬을 이용한 머신러닝, 딥러닝 실전 앱 개발</title>
      <link>http://example.com</link>
      <description>&lt;a href="http://example.com"&gt;이스케이프 처리 확인 전용 링크&lt;/a&gt;설명 입력</description>
      <book:publisher id="1">위키북스</book:publisher>
    </item>
  </channel>
</rss>

